In [1]:
import json, codecs, os
import sqlite3

In [2]:
nodes_fpath = os.getcwd() + '/raw_graph_schema/node_properties.json'
rels_fpath = os.getcwd() + '/raw_graph_schema/rel_properties.json'

In [3]:
from typing import List, Dict, Tuple
db_fpath =  '/home/22842219/Desktop/projects_data/spider/database'
sorted_schema = {}
checking_dbs = []
# {"department_management": {'db_id': str, 'tag_names': List(str), 'property_names': [(-1, "*"), (tag_idx, column_name0)...]
# tag_name:{propery_name: []}}}
for fpath in [nodes_fpath, rels_fpath]:
    with codecs.open(fpath, 'r', 'utf-8-sig') as f:
        data = json.load(f)
        for every in data:
            if 'nodeType' in every:
                tag = every['nodeType']
            elif 'relType' in every:
                tag = every['relType']
            tag_seps = tag.strip(':').strip("`").split('.')
            db_id = tag_seps[0]
            if db_id not in checking_dbs:
                checking_dbs.append(db_id)
                each_schema = {'db_id':db_id, 'tag_names_original': [], 'tag_names': [], \
                    'property_names': [(-1, "*")], 'property_types': []}
                sorted_schema[db_id]=each_schema
            target = sorted_schema[db_id]
            if db_id in target['db_id']:
                tag_name = tag_seps[1]
                if tag_name not in target:
                    target[tag_name]={}
                if tag_name not in target['tag_names_original']:
                    target['tag_names_original'].append(tag_name)
                    target['tag_names'].append(tag)


                if isinstance(every['propertyTypes'], list) and len(every['propertyTypes'])==1:
                    target['property_types'].append('_'.join(every['propertyTypes']))
                else:
                    target['property_types'].append(every['propertyTypes'])
                    
                # use sqlite cursor to get field values. 
                # we use the concept of picklist to save field values.
            # if db_id == 'department_management':
                db_path = os.path.join(db_fpath, db_id, '{}.sqlite'.format(db_id))
                tag_id = target['tag_names_original'].index(tag_name)
                target['property_names'].append((tag_id, every['propertyName']))
                if every['propertyName']:
                    fetch_sql = "SELECT DISTINCT `{}` FROM `{}`".format(every['propertyName'], tag_name)
                    print(db_id, tag_name, fetch_sql)
                    try:
                        conn = sqlite3.connect(db_path)
                        # conn.text_factory = bytes
                        c = conn.cursor()
                        c.execute(fetch_sql)
                        picklist = set()
                        # print(c.fetchall())
                        for x in c.fetchall():

                            if isinstance(x[0], str):
                                picklist.add(x[0])
                            elif isinstance(x[0], bytes):
                                try:
                                    picklist.add(x[0].decode("utf-8"))
                                except UnicodeDecodeError:
                                    picklist.add(x[0].decode("latin-1"))
                            else:
                                picklist.add(x[0])
                        picklist = list(picklist)
                    finally:
                        conn.close()
                    target[tag_name][every['propertyName']]=picklist
                
                    


flight_company airport SELECT DISTINCT `name` FROM `airport`
flight_company airport SELECT DISTINCT `IATA` FROM `airport`
flight_company airport SELECT DISTINCT `Country` FROM `airport`
flight_company airport SELECT DISTINCT `ICAO` FROM `airport`
flight_company airport SELECT DISTINCT `id` FROM `airport`
flight_company airport SELECT DISTINCT `City` FROM `airport`
flight_company operate_company SELECT DISTINCT `name` FROM `operate_company`
flight_company operate_company SELECT DISTINCT `id` FROM `operate_company`
flight_company operate_company SELECT DISTINCT `Type` FROM `operate_company`
flight_company operate_company SELECT DISTINCT `Principal_activities` FROM `operate_company`
flight_company operate_company SELECT DISTINCT `Incorporated_in` FROM `operate_company`
flight_company operate_company SELECT DISTINCT `Group_Equity_Shareholding` FROM `operate_company`
flight_company flight SELECT DISTINCT `id` FROM `flight`
flight_company flight SELECT DISTINCT `Pilot` FROM `flight`
flight_c

In [4]:

len(sorted_schema)


155

In [5]:
sorted_schema

{'flight_company': {'db_id': 'flight_company',
  'tag_names_original': ['airport', 'operate_company', 'flight'],
  'tag_names': [':`flight_company.airport`',
   ':`flight_company.operate_company`',
   ':`flight_company.flight`'],
  'property_names': [(-1, '*'),
   (0, 'name'),
   (0, 'IATA'),
   (0, 'Country'),
   (0, 'ICAO'),
   (0, 'id'),
   (0, 'City'),
   (1, 'name'),
   (1, 'id'),
   (1, 'Type'),
   (1, 'Principal_activities'),
   (1, 'Incorporated_in'),
   (1, 'Group_Equity_Shareholding'),
   (2, 'id'),
   (2, 'Pilot'),
   (2, 'airport_id'),
   (2, 'company_id'),
   (2, 'Vehicle_Flight_number'),
   (2, 'Velocity'),
   (2, 'Date'),
   (2, 'Altitude')],
  'property_types': ['String',
   'String',
   'String',
   'String',
   'Long',
   'String',
   'String',
   'Long',
   'String',
   'String',
   'String',
   'Double',
   'Long',
   'String',
   'Long',
   'Long',
   'String',
   'Double',
   'String',
   'Double'],
  'airport': {'name': ['Billund Airport',
    'Bergen Airport',
 

In [6]:
import os
outfpath= os.getcwd() +  '/schema.json'
with codecs.open(outfpath, 'w', encoding='utf-8') as out:
    json.dump(sorted_schema, out, indent=4)

In [7]:
sp_data_folder =  '/home/22842219/Desktop/phd/SemanticParser4Graph/sp_data_folder'
for split in ['train.json', 'dev.json']:
    fpth = os.path.join(sp_data_folder,  split )
    print(fpth)
    reformated_data = []
    with open(fpth, 'r', encoding='utf-8-sig') as f:
        data = json.load(f)
        for every in data:
            if every['answers'] and every['answers']!=[[0]]:
                reformated = {}
                reformated['question'] = every['question']
                reformated['query'] = every['cypher_query']
                reformated['db_id'] = every['db_id']
                reformated['answers'] = every['answers']
                reformated_data.append(reformated)

        with open(split, 'w') as out:
            json.dump(reformated_data, out, indent=4)

                    


/home/22842219/Desktop/phd/SemanticParser4Graph/sp_data_folder/train.json
/home/22842219/Desktop/phd/SemanticParser4Graph/sp_data_folder/dev.json
